In [1]:
import pandas as pd
import sklearn
from scipy import misc
import cv2
import numpy as np 

# path = '/Users/fsaxena/Documents/UCL/Masters/IntroML1/AMLSassignment19_20/AMLS_19-20_SN14002056'
path = '/home/fsaxena/amls/AMLSassignment19_20/AMLS_19-20_SN14002056'


assignment_celeb = 'celeba'
assignment_cartoon = 'cartoon_set'

In [2]:
def data_loader(path_to_root, assignment):
    return pd.read_csv(path_to_root + '/Datasets/original_dataset_AMLS_19-20/' + assignment + '/labels.csv', sep='\t')
    

In [3]:
def convert_image_to_vector(file):
    img = cv2.imread(file)
    return img

In [4]:
path_to_img=path+'/Datasets/original_dataset_AMLS_19-20/'+assignment_celeb+'/img/0.jpg'
path_pre_img = path+'/Datasets/original_dataset_AMLS_19-20/'+assignment_celeb+'/img/'

In [5]:
np.array(convert_image_to_vector(path_to_img))
convert_image_to_vector(path_to_img).shape

(218, 178, 3)

In [6]:
## Now need to add each raw pic to each part of the training set

In [7]:
celeb_dataframe = data_loader(path, assignment_celeb)
print(celeb_dataframe)

      Unnamed: 0  img_name  gender  smiling
0              0     0.jpg      -1        1
1              1     1.jpg      -1        1
2              2     2.jpg       1       -1
3              3     3.jpg      -1       -1
4              4     4.jpg      -1       -1
...          ...       ...     ...      ...
4995        4995  4995.jpg       1        1
4996        4996  4996.jpg       1        1
4997        4997  4997.jpg       1        1
4998        4998  4998.jpg       1        1
4999        4999  4999.jpg       1        1

[5000 rows x 4 columns]


In [8]:
vec_Array = []

In [9]:
for img_name in celeb_dataframe['img_name']:
    img_vec = np.array(convert_image_to_vector(path_pre_img + img_name))
    vec_Array.append(img_vec)

In [10]:
data_Dict = {
    'img_vec': vec_Array,
    'gender': celeb_dataframe['gender']
}

In [11]:
x_dataset = vec_Array
y_dataset = np.array(data_Dict['gender'])

In [12]:
# df = pd.DataFrame(data_Dict, columns = ['img_vec', 'gender'])

In [13]:
# df.head()

In [14]:
from sklearn.model_selection import train_test_split
 
x, X_test, y, y_test = train_test_split(
    x_dataset,
    y_dataset,
    test_size=0.2,
    shuffle=True,
    random_state=42,
)

In [15]:
X_train = x
y_train = y

In [16]:
# X_train, X_val, y_train, y_val = train_test_split(
#     x,
#     y,
#     test_size=0.25,
#     shuffle=True,
#     random_state=42,
# )

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from skimage import color

from skimage.feature import hog
from skimage.io import imread
from skimage.transform import rescale
 
class RGB2GrayTransformer(BaseEstimator, TransformerMixin):
    """
    Convert an array of RGB images to grayscale
    """
 
    def __init__(self):
        pass
 
    def fit(self, X, y=None):
        """returns itself"""
        return self
 
    def transform(self, X, y=None):
        """perform the transformation and return an array"""
        return np.array([color.rgb2gray(img) for img in X])
 
 
class HogTransformer(BaseEstimator, TransformerMixin):
    """
    Expects an array of 2d arrays (1 channel images)
    Calculates hog features for each img
    """
 
    def __init__(self, y=None, orientations=9,
                 pixels_per_cell=(8, 8),
                 cells_per_block=(3, 3), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
 
        def local_hog(X):
            return hog(X,
                       orientations=self.orientations,
                       pixels_per_cell=self.pixels_per_cell,
                       cells_per_block=self.cells_per_block,
                       block_norm=self.block_norm)
 
        try: # parallel
            return np.array([local_hog(img) for img in X])
        except:
            return np.array([local_hog(img) for img in X])

In [18]:
# create an instance of each transformer
grayify = RGB2GrayTransformer()
hogify = HogTransformer(
    pixels_per_cell=(8, 8),
    cells_per_block=(2,2),
    orientations=9,
    block_norm='L2-Hys'
)
scalify = StandardScaler()

In [19]:
# call fit_transform on each transform converting X_train step by step
X_train_gray = grayify.fit_transform(X_train)
X_train_hog = hogify.fit_transform(X_train_gray)
X_train_prepared = scalify.fit_transform(X_train_hog)
 
print(X_train_prepared.shape)

(4000, 19656)


In [20]:
sgd_clf = SGDClassifier(random_state=42, max_iter=1000, tol=1e-3, n_jobs = -1)
sgd_clf.fit(X_train_prepared, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [21]:
# X_val_gray = grayify.transform(X_val)
# X_val_hog = hogify.transform(X_val_gray)
# X_val_prepared = scalify.transform(X_val_hog)

X_test_gray = grayify.transform(X_test)
X_test_hog = hogify.transform(X_test_gray)
X_test_prepared = scalify.transform(X_test_hog)

In [22]:
y_test_pred = sgd_clf.predict(X_test_prepared)
print(np.array(y_test_pred == y_test)[:25])
print('')
print('Percentage correct: ', 100*np.sum(y_test_pred == y_test)/len(y_test))

[ True False False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True]

Percentage correct:  92.5


In [23]:
from sklearn.pipeline import Pipeline
from sklearn import svm
 
HOG_pipeline = Pipeline([
    ('grayify', RGB2GrayTransformer()),
    ('hogify', HogTransformer(
        pixels_per_cell=(8, 8),
        cells_per_block=(2,2),
        orientations=9,
        block_norm='L2-Hys')
    ),
    ('scalify', StandardScaler()),
    ('classify', SGDClassifier(random_state=42, max_iter=1000, tol=1e-3))
])
 
clf = HOG_pipeline.fit(X_train, y_train)
print('Percentage correct: ', 100*np.sum(clf.predict(X_test) == y_test)/len(y_test))

Percentage correct:  92.5


In [24]:
from sklearn.model_selection import GridSearchCV
 
param_grid = [
    {'hogify__orientations': [9],
    'hogify__cells_per_block': [(3, 3)],
    'hogify__pixels_per_cell': [(8, 8), (14, 14)]},
    {'hogify__orientations': [9],
     'hogify__cells_per_block': [(3, 3)],
     'hogify__pixels_per_cell': [(14, 14)],
     'classify': [
         SGDClassifier(random_state=42, max_iter=1000, tol=1e-3),
         svm.SVC(kernel='linear')]}
]

In [25]:
grid_search = GridSearchCV(HOG_pipeline,
                           param_grid,
                           cv=3,
                           n_jobs=-1,
                           scoring='accuracy',
                           verbose=1,
                           return_train_score=True)
 
grid_res = grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:  3.6min remaining:   42.9s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  3.8min finished


In [26]:
grid_res.best_estimator_

Pipeline(memory=None,
         steps=[('grayify', RGB2GrayTransformer()),
                ('hogify',
                 HogTransformer(block_norm='L2-Hys', cells_per_block=(3, 3),
                                orientations=9, pixels_per_cell=(8, 8),
                                y=None)),
                ('scalify',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('classify',
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=1000, n_iter_no_change=5, n_jobs=None,
                               penalty='l2', power_t=0.5, random_state=42,
                               shuffle=True, tol=0.001, validation_fraction=0.1,
                               verbose=0, warm_st

In [27]:
grid_res.best_score_

0.9035008002375408

In [28]:
grid_res.best_params_

{'hogify__cells_per_block': (3, 3),
 'hogify__orientations': 9,
 'hogify__pixels_per_cell': (8, 8)}

In [29]:
from sklearn.model_selection import RandomizedSearchCV

In [30]:
best_pred = grid_res.predict(X_test)
print('Percentage correct: ', 100*np.sum(best_pred == y_test)/len(y_test))

Percentage correct:  92.9
